In [1]:
from __future__ import division, print_function
from svd_improvement import *
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.sparse import csr_matrix
from scipy.linalg import svd, diagsvd
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
%matplotlib inline
seed = 417

In [2]:
ratings, users, movies = load_data()
# full matrix is relatively small, so it can be fitted in RAM
R = construct_full_matrix(ratings, users, movies, mode='dense')
train_mask, test_mask = get_train_test_masks(ratings, users, movies)

In [3]:
learning_rate = 0.01
n_epoch = 20
batch_size = 32
display_step = 1
K = 3
reg_U = 0.06
reg_V = 0.06

n_input = R.shape[1]
n_hidden_1 = 256

In [4]:
X = tf.placeholder("float", [None, n_input])
mask = tf.placeholder("float", [K+1, None, n_input])  # bool actually

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([n_input])),
}

In [5]:
def encoder(x):
    layer_1 = tf.add(tf.matmul(x, weights['encoder_h1']), 
                     biases['encoder_b1'])
    return layer_1


def decoder(x):
    layer_1 = tf.add(tf.matmul(x, weights['decoder_h1']), 
                     biases['decoder_b1'])
    return layer_1

In [6]:
k = 256
U, s, Vh = svds(csr_matrix(R * train_mask), k=k)
Sigma = np.zeros((U.shape[1], Vh.shape[0]))
Sigma[range(len(s)), range(len(s))] = s
V = np.dot(Vh.T, Sigma.T)

In [7]:
lambdas = np.ones(K+1)

In [8]:
choice_prob = 0.7
R_appr = np.dot(U, V.T)
av_error = rmse(R.values, R_appr)
rho_mask = np.random.rand(*R.shape) <= choice_prob
diff_mask = np.abs(R.values - R_appr) <= av_error
easy_pred_indices = np.logical_not(np.logical_xor(rho_mask, diff_mask))
nnz_entries = np.zeros(R.shape, dtype=bool)
nnz_entries[np.where(R)] = True
hard_pred_indices = np.logical_xor(nnz_entries[np.newaxis, :, :], divide_set(easy_pred_indices, lambdas.shape[0]-1))

In [9]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op
y_true = X


cost = tf.sqrt(tf.div(tf.reduce_sum(tf.pow(tf.multiply((y_true - y_pred)[np.newaxis, :, :], mask), 2), axis=(1, 2)), 
                      tf.reduce_sum(mask, axis=(1, 2))))
cost = tf.reduce_sum(tf.multiply(cost, tf.constant(lambdas, dtype='float')))
cost += reg_U * tf.reduce_sum(tf.pow(weights['encoder_h1'], 2))
cost += reg_V * tf.reduce_sum(tf.pow(weights['decoder_h1'], 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [132]:
R_train = R * train_mask

In [153]:
n_epoch = 30
batch_size = 32  # R.shape[0]

In [154]:
sess = tf.Session()
sess.run(init)
batch_count = int(math.ceil(R.shape[0] / batch_size))

for epoch in range(n_epoch):
    for i in range(batch_count):
        X_batch = R_train.iloc[i*batch_size : (i+1)*batch_size, :]
        mask_batch = hard_pred_indices[:, i*batch_size : (i+1)*batch_size, :]

        _, c = sess.run([optimizer, cost], feed_dict={X: X_batch, mask: mask_batch})
    # Display logs per epoch step
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1),
              "cost=", "{:.9f}".format(c))

print("Optimization Finished!")

Epoch: 0001 cost= 4178.850097656
Epoch: 0002 cost= 10.561882973
Epoch: 0003 cost= 3.312506676
Epoch: 0004 cost= 22.641895294
Epoch: 0005 cost= 2.073425770
Epoch: 0006 cost= 2.245977163
Epoch: 0007 cost= 19.672286987
Epoch: 0008 cost= 3.757085800
Epoch: 0009 cost= 32.208808899
Epoch: 0010 cost= 19.869768143
Epoch: 0011 cost= 37.960319519
Epoch: 0012 cost= 30.406936646
Epoch: 0013 cost= 15.409044266
Epoch: 0014 cost= 29.915733337
Epoch: 0015 cost= 20.637386322
Epoch: 0016 cost= 26.217214584
Epoch: 0017 cost= 33.569442749
Epoch: 0018 cost= 34.363590240
Epoch: 0019 cost= 32.727203369
Epoch: 0020 cost= 3.019398689
Epoch: 0021 cost= 33.077320099
Epoch: 0022 cost= 4.351412296
Epoch: 0023 cost= 31.440570831
Epoch: 0024 cost= 28.790927887
Epoch: 0025 cost= 2.116193295
Epoch: 0026 cost= 2.508628845
Epoch: 0027 cost= 36.567768097
Epoch: 0028 cost= 6.647939205
Epoch: 0029 cost= 2.296278954
Epoch: 0030 cost= 2.220875978
Optimization Finished!


In [161]:
tmp = sess.run(y_pred, feed_dict = {X: R})

In [169]:
rmse(R, tmp, mask=train_mask), rmse(R, np.dot(U, V.T), mask=train_mask)

(0.35026139, 0.33995953286733321)

In [170]:
rmse(R, tmp, mask=test_mask), rmse(R, np.dot(U, V.T), mask=test_mask)

(0.37770927, 0.64350642607939412)

In [136]:
sess.close()